In [1]:
import whisper

m = whisper.load_streaming_model("./.cache/whisper/stream-tiny.en.pt", strict=False)
print(m.transcribe_stream('./jfk.wav', chunk_size=32000))

RuntimeError: Model ./.cache/whisper/stream-tiny.en.pt not found; available models = ['tiny.en', 'tiny', 'base.en', 'base', 'small.en', 'small', 'medium.en', 'medium', 'large-v1', 'large-v2', 'large-v3', 'large', 'large-v3-turbo', 'turbo']

In [5]:
whisper_tiny = whisper.load_model("tiny.en")

In [8]:
print(whisper_tiny.dims)

ModelDimensions(n_mels=80, n_audio_ctx=1500, n_audio_state=384, n_audio_head=6, n_audio_layer=4, n_vocab=51864, n_text_ctx=448, n_text_state=384, n_text_head=6, n_text_layer=4)


In [10]:
from whisper.streaming import StreamingConfig

model_dims = whisper_tiny.dims
streaming_config = whisper.streaming.StreamingConfig()
model = whisper.WhisperStreaming(model_dims, streaming_config)

In [11]:
print(model)

WhisperStreaming(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 384, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(384, 384, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-3): 4 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=384, out_features=384, bias=True)
          (key): Linear(in_features=384, out_features=384, bias=False)
          (value): Linear(in_features=384, out_features=384, bias=True)
          (out): Linear(in_features=384, out_features=384, bias=True)
        )
        (attn_ln): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=1536, out_features=384, bias=True)
        )
        (mlp_ln): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_post): LayerNorm((384,), e

In [13]:
model.load_state_dict(whisper_tiny.state_dict(), strict=False)

_IncompatibleKeys(missing_keys=['decoder.blocks.0.p_choose_layer.energy_bias', 'decoder.blocks.0.p_choose_layer.q_energy_proj.layers.0.weight', 'decoder.blocks.0.p_choose_layer.q_energy_proj.layers.0.bias', 'decoder.blocks.0.p_choose_layer.q_energy_proj.layers.2.weight', 'decoder.blocks.0.p_choose_layer.q_energy_proj.layers.2.bias', 'decoder.blocks.0.p_choose_layer.q_energy_proj.layers.4.weight', 'decoder.blocks.0.p_choose_layer.q_energy_proj.layers.4.bias', 'decoder.blocks.0.p_choose_layer.q_energy_proj.layers.6.weight', 'decoder.blocks.0.p_choose_layer.q_energy_proj.layers.6.bias', 'decoder.blocks.0.p_choose_layer.k_energy_proj.layers.0.weight', 'decoder.blocks.0.p_choose_layer.k_energy_proj.layers.0.bias', 'decoder.blocks.0.p_choose_layer.k_energy_proj.layers.2.weight', 'decoder.blocks.0.p_choose_layer.k_energy_proj.layers.2.bias', 'decoder.blocks.0.p_choose_layer.k_energy_proj.layers.4.weight', 'decoder.blocks.0.p_choose_layer.k_energy_proj.layers.4.bias', 'decoder.blocks.0.p_choos